# `Process Data`

In [2]:
%matplotlib inline
%config IPython.matplotlib.backend = "retina"

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import rcParams

import transit
import emcee
import corner

import astroML 
from astroML.plotting import setup_text_plots
setup_text_plots(fontsize=14, usetex=True)

import astropy

from scipy import stats
import scipy.optimize as op
import scipy.signal as sig
from scipy.signal import argrelextrema, medfilt
from random import uniform, randrange

import kplr
from kplr.ld import get_quad_coeffs

import time
from collections import Counter, defaultdict, OrderedDict

import glob, os
import itertools

In [3]:
rcParams['figure.figsize'] = 16, 7
rcParams["figure.dpi"] = 150
rcParams["savefig.dpi"] = 150

In [4]:
pgf_with_latex = {
    "pgf.texsystem": "xelatex",         # use Xelatex which is TTF font aware
    "text.usetex": True,                # use LaTeX to write all text
    "axes.labelsize": 10,               # LaTeX default is 10pt font.
    "legend.fontsize": 8,               # Make the legend/label fonts a little smaller
    "xtick.labelsize": 8,
    "ytick.labelsize": 8,
    "text.latex.unicode": True,
    "pgf.preamble": [
        r'\usepackage{fontspec}',
        r'\setmainfont{Ubuntu}',
        r'\setmonofont{Ubuntu Mono}',
        r'\usepackage{unicode-math}'
        r'\setmathfont{Ubuntu}'
        r'\usepackage{amsmath}' #for \text command
    ]
}

rcParams.update(pgf_with_latex)

# Introduction

## Load the data

In [5]:
path_file = '/Users/mbadenas/Documents/Master UAB/Tesis UAB/TFM2018/clean_bat_files/LC_p13point5up/'
path_mergedLC = '/Users/mbadenas/Documents/Master UAB/Tesis UAB/TFM2018/merge_light_curves/LC_p13point5up_merged/'

properties_sample = pd.read_csv(path_file+'all_targets_P13point5up.csv', sep=',', comment='#')
targets = pd.read_csv(path_file + 'kepler_id.txt',delimiter=',', dtype=int, header=None, names=['kepid'])

In [6]:
df = pd.merge(targets, properties_sample, on=['kepid'], how='inner') 
sc_data = df.drop_duplicates('kepid') #remove duplicates (some systems have both sc and lc LC)

print("Check files:")

if (len(targets) != len(properties_sample)):
    print("*Lengths don't match:", len(targets), len(properties_sample))
    print("\tSome systems have both sc and lc data! Remove duplicates.")

if (len(sc_data) == len(targets)):
    print("*Lengths match:", len(sc_data), len(targets))
    print('\tDuplicates have been removed.\n A total of {} systems have *short-cadence* LC with SN > 7.1 for their first transit and only 1 planet'.format(len(sc_data)))
    
sc_data.shape

Check files:
*Lengths don't match: 55 984
	Some systems have both sc and lc data! Remove duplicates.
*Lengths match: 55 55
	Duplicates have been removed.
 A total of 55 systems have *short-cadence* LC with SN > 7.1 for their first transit and only 1 planet


(55, 141)

## Define the Systems

In [7]:
class System:    
    def __init__(self, kepid, rs, rs_err, smass, smass_err, teff, logg, feh, srho,
                time_BKJD, flux, flux_err, 
                t0, P, depth, b, duration, u1, u2, diffld, num_planets, incl, eccen, dor, sma,
                 transit_times,ind_missed_transits,
                rp, rp_solarRad, rp_err, teq, mp, 
                 trans_id, ftrans, ftrans_err, ttrans,dt_trans,
                res_transfit, P_distribution, MCMC_transits):
        
        """****** Kepler Reported Stellar Parameters ******"""
        self.kepid = kepid
        self.rs = rs # [Solar radii]. 
        self.rs_err = rs_err
        self.smass = smass # [Solar mass]
        self.smass_err = smass_err
        
        self.teff = teff;     
        self.logg = logg; # Stellar Surface Gravity [log10(cm/s**2)]  
        self.feh = feh;   
        self.srho = srho; #g/cm3
        
        """****** LC parameters ******"""
        self.time_BKJD = time_BKJD
        self.flux = flux
        self.flux_err = flux_err
        
        """****** Transit Parameters ******"""
        self.t0 = t0
        self.P = P;            
        self.depth = depth;  
        self.b = b; 
        self.duration = duration; 
        self.u1 = u1; 
        self.u2 = u2
        self.diffld = 0.0
        self.num_planets = num_planets
        self.incl = incl #[º]
        self.eccen = 0.0
        self.dor = dor #Planet-Star Distance over Star Radius
        self.sma = sma #Orbital semi-major axis [AU]
        
        self.transit_times = transit_times
        self.ind_missed_transits = ind_missed_transits
                   
        """****** Planetary Parameters ******"""
        self.rp = rp # [Earth radii]. This is "prad"  in the original file
        self.rp_solarRad = rp*0.009168 # in solar radii
        self.rp_err = rp_err
        self.teq = teq
        self.mp = 2.69*(rp)**(0.93) #Weiss & Marcy rel'n. It requires Rp in Earth radii
        
        """****** Internal Variables ******"""
        self.trans_id = np.empty(0)
        self.ftrans = np.empty(0)
        self.ftrans_err = np.empty(0)
        self.ttrans = np.empty(0)
        self.dt_trans = np.empty(0)
        self.res_transfit = np.empty(0)
        self.P_distribution = np.empty(0)
        self.MCMC_transits = np.empty(0)

## Characterize the Systems

Some initialization functions: `quadraticLD`calculates the quadratic LD coefficients of a star based on the model from Claret & Bloemen (2011). As for `fetchLC`, it simply loads the systems' light curves.

In [8]:
bjd_ref = 2454833

def quadraticLD(T, G, FEH): 
    """Confirm the quadratic LD coefficients with a model from Claret & Bloemen (2011).
    Use the Claret coefficients instead."""
    mu1, mu2 = get_quad_coeffs(T, G, FEH)
    return (mu1, mu2)

def fetchLC(name):
    id_kep = "%.0f" % name
    lc = path_mergedLC+('KID'+id_kep+'.txt')
    df = pd.read_csv(lc, sep="\t", skiprows=1, header=None, names=["Time BKJD", "Flux", "Flux_Err"])
    y = df['Flux']
    yerr = df['Flux_Err']
    x = df['Time BKJD']
    #print(np.median(y),np.mean(y),np.std(y))
    return (y, yerr, x)

Create the planet+star system. In this research, we will only evaluate systems with only 1 planet. 

In [9]:
targets = []

for row in sc_data.itertuples(index = True, name='Pandas'):
    kepid = getattr(row, "kepid")
    fluxLC, fluxLC_err, timeLC_BKJD = fetchLC(kepid)

    ## Store the stellar parameters
    rs = getattr(row, "koi_srad") #solar radii
    smass = getattr(row, "koi_smass")
    
    #Assuming that we have a 5% error on radius and mass (this is sensible thanks to Gaia)
    rs_err = rs*0.05
    smass_err = smass*0.05
    
    teff = getattr(row, "koi_steff")
    logg = getattr(row, "koi_slogg") # Stellar Surface Gravity [log10(cm/s**2)]  
    feh = getattr(row, "koi_smet")
    srho = getattr(row, "koi_srho") #g/cm3
    
    #### Store LC Parameters
    t0 = getattr(row, "koi_time0bk")
    P = getattr(row, "koi_period") # Orbital Period [days]
    depth = getattr(row, "koi_depth")/1e6
    b = getattr(row, "koi_impact")
    
    u1, u2 = quadraticLD(teff,logg,feh)

    num_planets = getattr(row, "koi_count")
    incl =  getattr(row, "koi_incl") #in degrees
    dor = getattr(row, "koi_dor")  #Planet-Star Distance over Star Radius
    
    duration = getattr(row, "koi_duration")/24. #in days. "Duration is measured from first contact between the planet and star until last contact."
    
    #Planetary parameters
    sma = getattr(row, "koi_sma")  #Orbit Semi-Major Axis [AU]
    sma_solarRad = sma*215
    teq = getattr(row, "koi_teq") 
    rp = getattr(row, "koi_prad") # in Earth radii
    rp_errPos = getattr(row, "koi_prad_err1"); rp_errNeg = getattr(row, "koi_prad_err2")
    rp_err = np.sqrt(rp_errPos**2+rp_errNeg**2)
        
    if (num_planets == 1):
        system = System(kepid, rs, rs_err, smass, smass_err, teff, logg, feh, srho,
                    timeLC_BKJD, fluxLC, fluxLC_err, 
                    t0, P, depth, b, duration, u1, u2, u1-u2, num_planets, incl, 0.0, dor, sma,
                        None, None,
                    rp, None, rp_err, teq, None, 
                        None, None, None, None, None,
                       None, None, None)

        targets.append(system)
                
    else:
        print('Star {:s} not stored (koi_count > 1)'.format(str(kepid)))

# Global Parameters

MCMC Global Initialization Parameters

In [12]:
sc = 58.0  #sc = 58 sec
tKep = sc/60/60/24  # sc in days
G = 6.6730e-11
steps, nwalkers = 3000, 100
steps_rho, nwalkers_rho = 2000, 100
snr_limit = 3

In [15]:
poly_order = 3
amplify = 2
print("Amplification of transit duration by:", amplify)

Amplification of transit duration by: 2


In [16]:
period_guess = 100 #randrange(14,500) #uniform(13.5, 500) print("Period guess [d]:", period_guess)

In [17]:
id_mcmc = ["Depth:", "Impact parameter (b):", "sigma:", "Stellar Mass (Ms, [Solar mass]):",
           "Stellar Radius (Rs, [Solar Radii]):", "Out-of-transit flux (f0):", "Orbital Period (P, [days]):", 
           "Time of transit (tc, [days]):", "Planetary Radius (Rp, [Rearth]):", "Rs_a:"]

header_mcmc = "This file shows the 50% percentile and the +- 1 sigma error interval of the parameters.\n"

header_rs = "This file shows the 50% percentile for the Stellar Radius (in solar radii), \
its +- 1 sigma error interval, and the MCMC acceptance fraction (in %). Note that these results \
are for the star's *observed* transits with a minimum SNR of {0:d}. \n".format(snr_limit)

# Transit Detection

In [18]:
def find_transits(star):
    ti = min(star.time_BKJD)
    tf = max(star.time_BKJD)
    
    n_min = int((ti-star.t0)/star.P)
    n_max = int((tf-star.t0)/star.P+1.)
    n = np.arange(n_min, n_max)
    
    t = star.t0+n*star.P
    t = t[t>ti] 
    t = t[t<tf]
    return(t)

In [19]:
def show_folded_lightcurve(star, plot=False):
    if plot==True:
        plt.figure(figsize=(15,6))
        plt.plot(star.dt_trans,star.ftrans,'.', ms = 2)     
        plt.title('Folded LC for KID'+str(star.kepid))
        plt.xlabel('Time from midtransit [d]'); plt.ylabel('Normalized Flux')
        plt.show(block=False)    
        time.sleep(0.2)
        plt.close()

def mark_transits(star, obs_trans, plot=False):
    if plot==True:
        plt.figure(figsize=(15,6))
        plt.plot(star.time_BKJD,star.flux,'.')        
        for j in range(len(star.transit_times)): 
            plt.axvline(star.transit_times[j], color='k', ls='-')
        for k in range(len(obs_trans)): 
            plt.axvline(obs_trans[k], color='r', ls='--')
        plt.title('KID'+str(star.kepid))
        plt.xlabel('Time BKJD [d]'); plt.ylabel('Normalized Flux')
        plt.show(block=False)    
        time.sleep(0.2)
        plt.close()
        
def show_transits(star, bad_trans, plot=False):        
    s1 = set(bad_trans)
    if plot==True:   
        for i in range(len(star.transit_times)):
            if (i not in bad_trans):
                plt.figure(figsize=(15,6))
                plt.title('Transit '+str(i+1))
                plt.plot(star.ttrans[star.trans_id==i]-star.transit_times[i],star.ftrans[star.trans_id==i], 'k.', ms = 2)     
                plt.xlabel('Time from midtransit [d]'); plt.ylabel('Normalized Flux')
                plt.show(block=False)    
                time.sleep(0.2)
                plt.close()

In [20]:
def get_data_in_transit(star, factor):
    window = factor*star.duration # In days
    total_points = 0
    
    for i in range(0,len(star.transit_times)):
        residual = star.time_BKJD - star.transit_times[i]
        points_in_transit = np.abs(residual) <= window
        total_points += np.sum(points_in_transit)

    time_in_transit = np.empty(total_points)
    flux_in_transit = np.empty(total_points)
    flux_err_in_transit = np.empty(total_points)
    flag_transit = np.empty(total_points)
    mid_trans = np.empty(total_points)

    total_points = 0

    for i in range(0, len(star.transit_times)):
        points_in_transit = np.abs(star.time_BKJD - star.transit_times[i]) <= window
        count_points = np.sum(points_in_transit)
        time_in_transit[(0 + total_points):(count_points + total_points)] = star.time_BKJD[points_in_transit]
        flux_in_transit[(0 + total_points):(count_points + total_points)] = star.flux[points_in_transit] #, star.flux_err[points_in_transit])[0]
        flux_err_in_transit[(0 + total_points):(count_points + total_points)] = star.flux_err[points_in_transit] #normalize(star.flux[points_in_transit], 
        flag_transit[(0 + total_points):(count_points + total_points)] = i
        mid_trans[(0 + total_points):(count_points + total_points)] = star.transit_times[i] 
        total_points += count_points
        
    star.trans_id = flag_transit #The counter starts at 0 (in other words, trans_id == 0 is already the first transit)
    star.ttrans = time_in_transit
    star.ftrans = flux_in_transit
    star.ftrans_err = flux_err_in_transit
    star.dt_trans = time_in_transit - mid_trans #if interested in the folded LC, plot target.dt_trans vs target.ftrans

In [21]:
def oot_fit(star, poly):
    transit_range = star.duration/2     
    
    if star.kepid == 8156120:
        poly = 5
    
    if star.kepid == 5812701:
        poly = 7
        
    if star.kepid == 12365184:
        transit_range += 0.08
    
    for i in range(len(star.transit_times)):
        dt = star.ttrans[star.trans_id == i] - star.transit_times[i]
        y = star.ftrans[star.trans_id==i]
        yerr = star.ftrans_err[star.trans_id==i]
        
        outside_trans = np.abs(dt) >= transit_range
        inside_trans = np.abs(dt) < transit_range
        num_oot_points = len(dt[outside_trans])
        
        if  (num_oot_points!= 0):
            coeffs = np.polyfit(dt[outside_trans], y[outside_trans], poly)
            baseline_fitted = np.polyval(coeffs, dt)
            
            newY = y/baseline_fitted
            newYerr = yerr/baseline_fitted 
                    
            fig = plt.figure()
            plt.plot(dt, y, 'mo', label = 'lc')
            plt.plot(dt[outside_trans], y[outside_trans], '.', label ='oot')
            plt.plot(dt, baseline_fitted, '.', label ='baseline')
            plt.plot(dt, newY, 'y-', label ='y/baseline')
            plt.legend()
            filepath_oot = '/Users/mbadenas/Documents/Master UAB/Tesis UAB/TFM2018/oot_fit/'
            store_oot = "oot"+str(star.kepid)+"_trans"+str(i+1)+"_poly"+str(poly)+".png"
            fig.savefig(filepath_oot+store_oot)
            #plt.show(block=False)    
            #time.sleep(0.1)
            plt.close(fig)
            
            star.ftrans[star.trans_id==i] = newY
            star.ftrans_err[star.trans_id==i] = newYerr

In [22]:
def find_missed_transits(star):
    gaps = []
    for i in range(len(star.transit_times)):
        if len(star.ttrans[star.trans_id==i])==0: 
            gaps.append(i)
    missing_transits = sorted(gaps, reverse=True)
    return(missing_transits)

In [23]:
def export_folded_lc(system):
    lc = pd.DataFrame(OrderedDict({'Time': system.time_BKJD, 'Flux': system.flux, 'Flux_Err': system.flux_err}))
     
    np.savetxt("/Users/mbadenas/Documents/Master UAB/Tesis UAB/TFM2018/folded_LC/"+str(system.kepid)+".txt", 
               lc.values, fmt='%f', delimiter="\t") 

The code below performs two functions: 
    1. It identifies the light curves that don't have a visible transit. (With the data being used as of June 30, this amounts to 4 systems).
    2. For the remaining light curves, it calculates all the expected transits and then looks at which transits are actually visible (sometimes, there are gaps in the data and transits are lost). 

In [24]:
stars_wo_transits = []

In [25]:
for target in targets:
    print("\n******************************** KID"+str(target.kepid)+" ********************************\n")
    target.transit_times = find_transits(target)    
    get_data_in_transit(target, amplify)

    if (len(target.transit_times)==0): stars_wo_transits.append(target.kepid)

    target.ind_missed_transits = find_missed_transits(target) # On notation: Imagine target.ind_missed_transits = [18]. This means 
                                                              # that the transit with flag_id = 18 hasn't been observed (and not the transit 
                                                              # with flag_id = 17)
    oot_fit(target, poly_order)
    observed_transits = np.delete(target.transit_times, target.ind_missed_transits)
    mark_transits(target, observed_transits, False)
    show_transits(target, target.ind_missed_transits, False)

    if len(observed_transits!=0): show_folded_lightcurve(target, False)

    export_folded_lc(target)

    print("Transit duration: {0:0.5} [d]".format(target.duration))
    print("Period: {0:s} [d]".format(str(target.P)))
    print("Estimated transits at ({0:s}):".format(str(len(target.transit_times))), target.transit_times)
    print("Observed transits at ({0:s}):".format(str(len(observed_transits))), observed_transits)
    print("Index of missed transits:", target.ind_missed_transits)


******************************** KID11133306 ********************************

Transit duration: 0.19219 [d]
Period: 41.74598855 [d]
Estimated transits at (26): [ 544.3670594   586.11304795  627.8590365   669.60502505  711.3510136
  753.09700215  794.8429907   836.58897925  878.3349678   920.08095635
  961.8269449  1003.57293345 1045.318922   1087.06491055 1128.8108991
 1170.55688765 1212.3028762  1254.04886475 1295.7948533  1337.54084185
 1379.2868304  1421.03281895 1462.7788075  1504.52479605 1546.2707846
 1588.01677315]
Observed transits at (25): [ 544.3670594   586.11304795  627.8590365   669.60502505  711.3510136
  753.09700215  794.8429907   836.58897925  878.3349678   920.08095635
  961.8269449  1003.57293345 1045.318922   1087.06491055 1128.8108991
 1170.55688765 1212.3028762  1254.04886475 1337.54084185 1379.2868304
 1421.03281895 1462.7788075  1504.52479605 1546.2707846  1588.01677315]
Index of missed transits: [18]

******************************** KID3733628 **************

Transit duration: 0.20198 [d]
Period: 29.90721859 [d]
Estimated transits at (1): [773.4377418]
Observed transits at (1): [773.4377418]
Index of missed transits: []

******************************** KID11027624 ********************************

Transit duration: 1.0045 [d]
Period: 394.6249514 [d]
Estimated transits at (0): []
Observed transits at (0): []
Index of missed transits: []

******************************** KID5009743 ********************************

Transit duration: 0.2377 [d]
Period: 41.6980186 [d]
Estimated transits at (1): [878.4417962]
Observed transits at (1): [878.4417962]
Index of missed transits: []

******************************** KID2853093 ********************************

Transit duration: 0.15306 [d]
Period: 161.5278347 [d]
Estimated transits at (1): [1005.6405435]
Observed transits at (1): [1005.6405435]
Index of missed transits: []

******************************** KID6442377 ********************************

Transit duration: 0.27813 [d]
Period: 30.22910274 

Transit duration: 0.21007 [d]
Period: 38.47876629 [d]
Estimated transits at (27): [ 267.28911087  305.76787716  344.24664345  382.72540974  421.20417603
  459.68294232  498.16170861  536.6404749   575.11924119  613.59800748
  652.07677377  690.55554006  729.03430635  767.51307264  805.99183893
  844.47060522  882.94937151  921.4281378   959.90690409  998.38567038
 1036.86443667 1075.34320296 1113.82196925 1152.30073554 1190.77950183
 1229.25826812 1267.73703441]
Observed transits at (11): [ 267.28911087  305.76787716  344.24664345  382.72540974  421.20417603
  536.6404749   575.11924119  613.59800748 1190.77950183 1229.25826812
 1267.73703441]
Index of missed transits: [23, 22, 21, 20, 19, 18, 17, 16, 15, 14, 13, 12, 11, 10, 6, 5]

******************************** KID3835670 ********************************

Transit duration: 0.32351 [d]
Period: 14.55730897 [d]
Estimated transits at (38): [261.55348376 276.11079273 290.6681017  305.22541067 319.78271964
 334.34002861 348.89733758 363.4

Transit duration: 0.1955 [d]
Period: 14.00640392 [d]
Estimated transits at (31): [268.26648528 282.2728892  296.27929312 310.28569704 324.29210096
 338.29850488 352.3049088  366.31131272 380.31771664 394.32412056
 408.33052448 422.3369284  436.34333232 450.34973624 464.35614016
 478.36254408 492.368948   506.37535192 520.38175584 534.38815976
 548.39456368 562.4009676  576.40737152 590.41377544 604.42017936
 618.42658328 632.4329872  646.43939112 660.44579504 674.45219896
 688.45860288]
Observed transits at (24): [268.26648528 282.2728892  296.27929312 310.28569704 324.29210096
 338.29850488 352.3049088  366.31131272 380.31771664 394.32412056
 408.33052448 422.3369284  436.34333232 506.37535192 520.38175584
 534.38815976 548.39456368 562.4009676  576.40737152 590.41377544
 604.42017936 618.42658328 674.45219896 688.45860288]
Index of missed transits: [28, 27, 26, 16, 15, 14, 13]

******************************** KID6707835 ********************************

Transit duration: 0.17285 [d]

In [26]:
print("There are {:s} stars without transits. These targets won't be studied".format(str(len(stars_wo_transits))))
print(stars_wo_transits)

There are 4 stars without transits. These targets won't be studied
[11027624, 11622600, 7761545, 10795103]


In [27]:
print(steps)
print(steps_rho)

3000
2000


# Fit of the Stellar Density

## Transit Routine (DFM)

A Python library for generating light curves of transiting planets. See https://github.com/dfm/transit/blob/master/transit/transit.py for original code

In [28]:
def lnprior_rho(theta, maxb):
    pdepth, pb, sigma, pradius, f0, ptc = theta 
    if ((0 <= pb < maxb) and (0 <= sigma) and (pradius > 0) and (pdepth > 0) and (ptc**2<0.01)):
        return 0.0
    return -np.inf  

def lnlike_rho(theta, timeLC, fluxLC, errorLC, allfixed):
    ecc,  mass, masserr, tKep, u1, u2, maxb, P = allfixed
    pdepth, pb, sigma, pradius, f0, ptc = theta 
    
    s = transit.System(transit.Central(mu1=u1, mu2=u2, mass = mass, radius = pradius))
    body = transit.Body(radius=np.sqrt(pdepth)*pradius, period=P, t0=ptc, b=np.abs(pb), e=ecc)
    s.add_body(body)
    inv_sigma2 = 1.0/(errorLC**2 + sigma**2)
    try: 
        ftheo = s.light_curve(timeLC, texp=tKep, tol=1e-08, maxdepth=5)
    except ValueError:
        return -np.inf
    ftheo = ftheo-1+f0
    chi2 = -0.5*(np.sum((fluxLC-ftheo)**2*inv_sigma2 - np.log(inv_sigma2)))
    return chi2
                 
def lnprob_rho(theta, timeLC, fluxLC, errorLC, allfixed):
    ecc, mass, masserr, tKep, u1, u2, maxb, P = allfixed
    lp_rho = lnprior_rho(theta, maxb)
    if not np.isfinite(lp_rho):
        return -np.inf
    return lp_rho + lnlike_rho(theta, timeLC, fluxLC, errorLC, allfixed)

## MCMC

In [29]:
def fit_rho(star, t, f, ferr, num_trans, showLC, showMCMC):
    
    max_b = 1+star.rp_solarRad/star.rs
    print("\nMax b (rp in Solar Radius):", max_b)
    
    allfixed_rho = np.array([star.eccen, star.smass, 0.0, tKep, star.u1, star.u2, max_b, star.P])
    first_guess =  np.array([star.depth/1.2, star.b, 0.0, star.rs, 1.0, 0.0])
    
    nll_rho = lambda *args: -lnprob_rho(*args)
       
    result = op.minimize(nll_rho, first_guess, args=(t, f, ferr, allfixed_rho), 
                       options={'maxiter':1e5,'disp': True},
                       method='Nelder-Mead') 
    
    depth_ml, b_ml, sigma_ml, radius_ml, f0_ml, tc_ml = result["x"]
    
    # Compute the theoretical light curve integrated over a Kepler short-cadence exposure time.
    s = transit.System(transit.Central(mu1=star.u1, mu2=star.u2, mass=star.smass, radius=radius_ml))
    body = transit.Body(radius=np.sqrt(depth_ml)*radius_ml,period=star.P, t0=tc_ml,b=np.abs(b_ml),e=star.eccen)
    s.add_body(body)
    t_fit = np.arange(-1, 1, tKep*0.01)
    f_fit = s.light_curve(t_fit, texp=tKep, tol=1e-08, maxdepth=5)
    f_fit = f_fit - 1.0 + f0_ml
    
    if showLC == True:
        fig = plt.figure(figsize=(16,7))
        plt.title(str(star.kepid)+'. Stellar Density Fit for Transit '+str(num_trans+1), fontsize = 15)
        plt.plot(t, f, '.')
        plt.plot(t_fit, f_fit, color='r', lw = 3)
        plt.xlabel('Time from midtransit [days]')
        plt.ylabel('Normalized flux')
        plt.xlim([min(t),max(t)]) 
        plt.show(block=False)    
        time.sleep(0.2)
        plt.close()
       
    ndim_rho = len(result["x"])
    
    p0 = [result["x"]*(1+1e-8*np.random.randn(ndim_rho)) for i in range(nwalkers_rho)]
    sampler = emcee.EnsembleSampler(nwalkers_rho, ndim_rho, lnprob_rho, args=(t, f, ferr, allfixed_rho))
    
    print("Running burn-in")
    pos, prob, state = sampler.run_mcmc(p0, 200, progress=True);
    sampler.reset() # Reset the chain to remove the burn-in samples.
    print("\n------ Run MCMC -------")
    sampler.run_mcmc(p0, steps_rho, rstate0 = state, progress=True);
    samples = sampler.chain[:, int(steps_rho/10):, :].reshape((-1, ndim_rho)) #burnin currently set to 10%
    
    samples[:, 2] = np.exp(samples[:, 2])
    
    #Get the 50% percentile and the +- 1 sigma error of the parameters and 2 derived quantities 
    depth_mcmc, b_mcmc, sigma_mcmc, \
    rs_mcmc, f0_mcmc, tc_mcmc = map(lambda v: (v[1], v[2]-v[1], v[1]-v[0]), 
                               zip(*np.percentile(samples, [15.86, 50, 84.14], axis=0)))
    
    if showMCMC == True:
        print("\n------ Post-MCMC Results ------")
        print("\nDepth:\n\tTrue = {:0.3f}".format(star.depth))
        print("\tMCMC Fit = {0:0.3f} (+{1:0.3f},-{2:0.3f})".format(depth_mcmc[0],depth_mcmc[1],depth_mcmc[2]))
        print("\nImpact parameter:\n\tTrue = {:0.3f}".format(star.b))
        print("\tMCMC Fit = {0:0.3f} (+{1:0.3f},-{2:0.3f})".format(b_mcmc[0],b_mcmc[1],b_mcmc[2]))
        print("\nStellar radius [Rsun]:\n\tREPORTED = {:0.3f}".format(star.rs))
        print("\tMCMC Fit = {0:0.3f} (+{1:0.3f},-{2:0.3f})".format(rs_mcmc[0], rs_mcmc[1],rs_mcmc[2]))
   
    #---------- Plot the LC with the MCMC fit ----------
    s = transit.System(transit.Central(mu1=star.u1, mu2=star.u2, mass=star.smass, radius=rs_mcmc[0]))
    body = transit.Body(radius=np.sqrt(depth_mcmc[0])*rs_mcmc[0], period=star.P, t0=tc_mcmc[0], 
                        b=np.abs(b_mcmc[0]), e=star.eccen)
    s.add_body(body)
    t_fit = np.arange(-1, 1, tKep*0.01)
    f_fit = s.light_curve(t_fit, texp=tKep, tol=1e-08, maxdepth=4)
    f_fit = f_fit - 1.0 + f0_mcmc[0]
    fig_lc = plt.figure(figsize=(15,4))
    plt.title('MCMC Fit for KID'+str(star.kepid))
    plt.plot(t, f, '.')
    plt.plot(t_fit, f_fit, 'r', lw = 3)
    plt.xlabel('Time [d]'); plt.ylabel('Normalized Flux')
    plt.xlim([min(t),max(t)])
    
    filepath_rho = '/Users/mbadenas/Documents/Master UAB/Tesis UAB/TFM2018/results/new_rs/'
    store_lcfit = "fit"+str(star.kepid)+"_st"+str(steps_rho)+"_trans"+str(num_trans+1)+".png"
    fig_lc.savefig(filepath_rho+store_lcfit)
    plt.close(fig_lc)
        
    true_params = [star.depth, star.b, None, star.rs, 1.0, 0.0]
    res_labels = [r'Depth', r'$b$', r'$\sigma$', r'$R_{s} [R_{\odot}]$', r'$f_{0}$', r'$t_{c}$']
    
    fig_corner = corner.corner(samples, weights = None, labels = res_labels, quantiles=[0.16, 0.5, 0.84],
                               truths = true_params, show_titles = True, title_args={"fontsize": 12}, 
                               plot_contours=True, truth_color='r', plot_datapoints=True) 
        
    plt.suptitle("KID"+str(star.kepid)+". Corner Plot for Transit "+str(num_trans+1), fontsize = 20)
    store_corner = "corner"+str(star.kepid)+"_st"+str(steps_rho)+"_trans"+str(num_trans+1)+".png"
    fig_corner.savefig(filepath_rho+store_corner)
    plt.close(fig_corner)
   
    #Acceptance fraction: s a rule of thumb, the acceptance fraction (af) should be between 0.2 and 0.5. 
    #If af < 0.2 decrease the a parameter. If af > 0.5 increase the a parameter. 
    af = sampler.acceptance_fraction  
    af_percentage = 100*np.mean(sampler.acceptance_fraction)
    
    print("Mean acceptance fraction:", np.mean(af))
    print("Acceptance fraction: {0:.2f} %".format(af_percentage))
    
    #Steps plot
    fig_steps, axes = plt.subplots(ndim_rho, 1, sharex=True, figsize=(8, 9))
    axes[0].plot(sampler.chain[:, :, 0].T, ls='dotted', color = "xkcd:sky blue") #depth = 0
    axes[0].set_ylabel(r'Depth'); axes[0].axhline(true_params[0], color='red')
    axes[1].plot(sampler.chain[:, :, 1].T,'.', ls='dotted', color="xkcd:sky blue")
    axes[1].set_ylabel(r'$b$'); axes[1].axhline(true_params[1], color='red')
    axes[2].plot((sampler.chain[:, :, 2]).T, ls='dotted', color="xkcd:sky blue")
    axes[2].set_ylabel(r'$\sigma$');
    axes[3].plot((sampler.chain[:, :, 3]).T, ls='dotted', color="xkcd:sky blue")
    axes[3].set_ylabel(r'$R_{s} [R_{\odot}]$'); axes[3].axhline(true_params[3], color='red')
    axes[4].plot((sampler.chain[:, :, 4]).T, ls='dotted', color="xkcd:sky blue")
    axes[4].set_ylabel(r'$f_{0}$'); axes[4].axhline(true_params[4], color='red')
    axes[5].plot((sampler.chain[:, :, 5]).T, ls='dotted',color="xkcd:sky blue")
    axes[5].set_ylabel(r"$t_{c}$"); axes[5].axhline(true_params[5], color='red')
    axes[5].set_xlabel("Step number")
    fig_steps.tight_layout(h_pad=0.0)
    store_steps= "steps"+str(star.kepid)+"_st"+str(steps_rho)+"_trans"+str(num_trans+1)+".png"
    fig_steps.savefig(filepath_rho+store_steps)
    plt.close(fig_steps)
    
    results_mcmc = [depth_mcmc, b_mcmc, sigma_mcmc, rs_mcmc, f0_mcmc, tc_mcmc]
    rs_results = np.array([rs_mcmc[0],rs_mcmc[1],rs_mcmc[2]])
    
    return radius_ml, rs_results, af_percentage

Store the KID of stars with missing or bad (SNR < 3) transits 

In [ ]:
no_transits = set(stars_wo_transits)
ugly_stars = defaultdict(list)

for i in range(len(stars_wo_transits)):
    idx_target = stars_wo_transits[i]
    ugly_stars[idx_target] = []
    
print(ugly_stars)

defaultdict(<class 'list'>, {11027624: [], 11622600: [], 7761545: [], 10795103: []})


## Fit individual Transits

Fit the stellar radius to obtain a better estimate of the stellar density

In [ ]:
for target in targets:
    print("\n******************************** KID"+str(target.kepid)+" ********************************")
    plt.close("all")
    key = target.kepid
    missed_trans = set(target.ind_missed_transits)

    rs_trans_fit = []
    rs_trans_afrac = []
        
    for i in range(0, len(target.transit_times)): 
        if i in missed_trans:
            ugly_stars[key].append(i)
        else: 
            print("\n\t*TRANSIT "+str(i+1))
            dt_i = target.ttrans[target.trans_id==i]-target.transit_times[i]
            f_trans_i = target.ftrans[target.trans_id==i]

            oot = np.abs(dt_i)>=(target.duration/2)
            iit = np.abs(dt_i)<(target.duration/2)
            d = 1-np.mean(f_trans_i[iit])
            error_trans_oot = np.std(f_trans_i[oot])
            snr = d/error_trans_oot
            
            print("\t\tSignal = {0:0.5f}. Noise (rms) = {1:0.5f}. SNR = {2:0.5f}".format(d, error_trans_oot, snr))
            print("\t\tOut-of-transit error of the transit LC = {0:0.5f}".format(error_trans_oot))

            if snr < snr_limit or np.isnan(snr): 
                ugly_stars[key].append(i)
            else:
                rsFit, rsMCMC, rs_afraction = fit_rho(target, dt_i,  f_trans_i, 0.1*error_trans_oot, i, False, False)
                
                print("\tStellar Radius [Rsun]:\tMCMC = {0:0.5f}.\t1st Fit = {1:0.5f}.\tKepler = {2:0.5f}".format(rsMCMC[0], rsFit, target.rs))
                print("\tError Stellar Radius [Rsun]:\tMCMC (up) = {0:0.5f}, (down) = {1:0.5f}.\tOld = {2:0.5f}".format(rsMCMC[1], rsMCMC[2],  target.rs_err))
                
                rs_trans_fit.append(rsMCMC)
                rs_trans_afrac.append(rs_afraction)
    
    if len(rs_trans_fit) != 0:
        locs_rsNew = '/Users/mbadenas/Documents/Master UAB/Tesis UAB/TFM2018/results/new_rs/'
        fname_rsNew = str(target.kepid)+"_st"+str(steps_rho)+"_RsMCMC.txt"
        np.savetxt(locs_rsNew+fname_rsNew, np.column_stack((rs_trans_fit, rs_trans_afrac)), 
                       fmt = "%0.3f", header = header_rs)


******************************** KID11133306 ********************************

	*TRANSIT 1
		Signal = 0.00038. Noise (rms) = 0.00030. SNR = 1.27823
		Out-of-transit error of the transit LC = 0.00030

	*TRANSIT 2
		Signal = 0.00039. Noise (rms) = 0.00029. SNR = 1.34154
		Out-of-transit error of the transit LC = 0.00029

	*TRANSIT 3
		Signal = 0.00036. Noise (rms) = 0.00028. SNR = 1.25945
		Out-of-transit error of the transit LC = 0.00028

	*TRANSIT 4
		Signal = 0.00034. Noise (rms) = 0.00028. SNR = 1.21716
		Out-of-transit error of the transit LC = 0.00028

	*TRANSIT 5
		Signal = 0.00024. Noise (rms) = 0.00028. SNR = 0.85699
		Out-of-transit error of the transit LC = 0.00028

	*TRANSIT 6
		Signal = 0.00032. Noise (rms) = 0.00029. SNR = 1.10424
		Out-of-transit error of the transit LC = 0.00029

	*TRANSIT 7
		Signal = 0.00036. Noise (rms) = 0.00028. SNR = 1.26480
		Out-of-transit error of the transit LC = 0.00028

	*TRANSIT 8
		Signal = 0.00038. Noise (rms) = 0.00029. SNR = 1.28654
		Ou

/Users/mbadenas/anaconda3/lib/python3.6/site-packages/transit/transit.py:518: RuntimeWarning: divide by zero encountered in log
  np.log(self.central.dilution)-np.log(1.0-self.central.dilution)


Optimization terminated successfully.
         Current function value: -7291.683701
         Iterations: 322
         Function evaluations: 526


  0%|          | 0/200 [00:00<?, ?it/s]

Running burn-in


...

100%|██████████| 2000/2000 [10:10<00:00,  3.45it/s]


Mean acceptance fraction: 0.47151499999999996
Acceptance fraction: 47.15 %
	Stellar Radius [Rsun]:	MCMC = 0.71829.	1st Fit = 0.71660.	Kepler = 0.87900
	Error Stellar Radius [Rsun]:	MCMC (up) = 0.00410, (down) = 0.00187.	Old = 0.04395

	*TRANSIT 25
		Signal = 0.01853. Noise (rms) = 0.00099. SNR = 18.74872
		Out-of-transit error of the transit LC = 0.00099

Max b (rp in Solar Radius): 1.138302252559727
Optimization terminated successfully.
         Current function value: -7444.587406
         Iterations: 190
         Function evaluations: 307
Running burn-in


100%|██████████| 200/200 [00:59<00:00,  3.67it/s]



------ Run MCMC -------


 88%|████████▊ | 1750/2000 [08:24<01:12,  3.46it/s]

## Fit the folded LC

For the star 5812701, the oot region of 3 individual transits were not fitted properly when applying the 3rd order polynomial. To avoid them destroying the beauty of the folded light curve plotted in the pie chart, I've added these 3 bad transits to the `missed_transits` list. Originally I was doing this *just* done for visual purposes under Section 8, but I moved this here so that I could obtain a proper fit of the folded LC.

In [ ]:
for target in targets:
    if target.kepid == 5812701:
        bad_tr = [23,49,65]
        target.ind_missed_transits.extend(bad_tr)
        missed_trans = set(target.ind_missed_transits)
        print(target.ind_missed_transits)
        for i in range(len(target.transit_times)): 
            if i not in missed_trans:
                plt.plot(target.ttrans[target.trans_id==i]-target.transit_times[i], target.ftrans[target.trans_id==i],'.', 
                       color = 'royalblue', ms = 0.3)

In [ ]:
full_lc_targets = {2987027, 4742414, 5383248, 5812701, 11391018, 11449844, 12365184, 12403119}

for target in targets:
    if target.kepid in full_lc_targets:
        print("\n******************************** KID"+str(target.kepid)+" ********************************")
        
        oot = np.abs(target.dt_trans)>=(target.duration/2)
        error_full_lc = np.std(target.ftrans[oot])
        
        rs_full_fit, rs_full_MCMC, rs_full_afrac = fit_rho(target, target.dt_trans,  
                                                               target.ftrans, 0.1*error_full_lc, i, False, False)
        
        print("rs_full_fit:", rs_full_fit)
        print("rs_full_afrac:", rs_full_afrac)

# Update Rs

Update stellar radius.

In [ ]:
path_newRs = '/Users/mbadenas/Documents/Master UAB/Tesis UAB/TFM2018/results/new_rs'
os.chdir(path_newRs)

modified_rs = 0
for target in targets:
    for file in glob.glob(str(target.kepid)+"*[!\BAD].txt"):
        print("------------------------KID{0:d}------------------------".format(target.kepid))
        
        data = pd.read_csv(file, delimiter=" ", names = ['Rs', '+err', '-err', 'afrac'],
                           header = 1, na_values = '#N/A')
        
        data_mean = data.mean()
        rs_new = data_mean['Rs']
        rs_err_new = np.sqrt(data_mean['+err']**2+data_mean['-err']**2)
        
        print("\tOld R*: {0:0.3f} ({1:0.3f}) [Rsun]\
        \n\tNew R*: {2:0.3f} ({3:0.3f}) [Rsun]\n".format(target.rs, target.rs_err, 
                                                       rs_new, rs_err_new))
        modified_rs += 1 

        
        target.rs = rs_new
        target.rs_err = rs_err_new  
        
        
print("The stellar radius has been modified for {0:d} systems.".format(modified_rs))

# Global Fit

## Transit Routine (DFM)

A Python library for generating light curves of transiting planets. See https://github.com/dfm/transit/blob/master/transit/transit.py for original code

In [ ]:
def ln_probtrans(P, rstar, mstar):
    alog = (1/3)*np.log(mstar)+(2/3)*np.log(P)
    prob = np.log(rstar)-alog
    return(prob)

In [ ]:
def lnlike(theta, timeLC, fluxLC, errorLC, allfixed):
    """
    Calculates the log of the likelihood of the transit model being the right model given the following parameters:
    theta[0] = pdepth = (Rp/Rs)^2 #b=0 centre of stellar disk & b=1 at the cusp of the disc
    theta[1] = pb = the mean impact parameter, measured in stellar radii (not Solar radii). See documentation.
    theta[2] = sigma = an additional white noise term
    theta[3] = pmass = the mass of the star (controlled via gaussian prior)
    theta[4] = pradius = the radius of the star (controlled via gaussian prior)
    theta[5] = f0 = the out of eclipse flux
    theta[6] = pperiod = Orbital period 
    theta[7] = ptc = Transit time (time of conjunction) 
    
    Note: pmass and prad (stellar mass and radius) controlled via gaussian prior. 
    By letting M* and R* be free parameters in the model, we allow the stellar density to fluctuate. 
    """
    ecc, mass, masserr, radius, radiuserr, tKep, u1, u2, maxb = allfixed
    pdepth, pb, sigma, pmass, pradius, f0, pperiod, ptc = theta 
    
    s = transit.System(transit.Central(mu1=u1, mu2=u2, mass = pmass, radius = pradius))
    body = transit.Body(radius=np.sqrt(pdepth)*pradius, period=pperiod, t0=ptc, b=np.abs(pb), e=ecc)
    s.add_body(body)
    inv_sigma2 = 1.0/(errorLC**2 + sigma**2)
    
    try: 
        ftheo = s.light_curve(timeLC, texp=tKep, tol=1e-08, maxdepth=4)
    except ValueError:
        return -np.inf
    
    ftheo = ftheo-1+f0
    
    return -0.5*(np.sum((fluxLC-ftheo)**2*inv_sigma2 - np.log(inv_sigma2)) + ((pmass-mass)/masserr)**2
            + ((pradius-radius)/radiuserr)**2)

In [ ]:
def lnprior(theta, maxb):
    ##https://github.com/stan-dev/stan/wiki/Prior-Choice-Recommendations
    pdepth, pb, sigma, pmass, pradius, f0, pperiod, ptc = theta
    if ((0 <= pb < maxb) and (0 <= sigma) and (pradius > 0) 
        and (pdepth > 0) and (pmass > 0.0) and (13.5 < pperiod)  and (ptc**2<0.01)):
        return ln_probtrans(pperiod, pradius, pmass)
    return -np.inf  

def lnprior_MCMC(theta, maxb): ##https://github.com/stan-dev/stan/wiki/Prior-Choice-Recommendations
    pdepth, pb, sigma, pmass, pradius, f0, pperiod, ptc = theta
    if ((0 <= pb < maxb) and (0 <= sigma) and (pradius > 0) 
        and (pdepth > 0) and (pmass > 0.0) and (13.5 < pperiod)  and (ptc**2<0.01)):
        return ln_probtrans(pperiod, pradius, pmass) 
    return -np.inf    

In [ ]:
def lnprob(theta, timeLC, fluxLC, errorLC, allfixed):
    ecc, mass, masserr, radius, radiuserr, tKep, u1, u2, maxb = allfixed
    lp = lnprior(theta, maxb)
    if not np.isfinite(lp):
        return -np.inf
    return lp + lnlike(theta, timeLC, fluxLC, errorLC, allfixed)

def lnprob_MCMC(theta, timeLC, fluxLC, errorLC, allfixed_MCMC):
    ecc, mass, masserr, radius, radiuserr, tKep, u1, u2, maxb = allfixed_MCMC
    lp_MCMC = lnprior_MCMC(theta, maxb)
    if not np.isfinite(lp_MCMC):
        return -np.inf
    return lp_MCMC + lnlike(theta, timeLC, fluxLC, errorLC, allfixed_MCMC)

In [ ]:
period_guess, steps

In [ ]:
def run_transit_routine(star, t, f, ferr, transit_num, showLC):
    depth_guess = 1-min(f)
    rp_guess = np.sqrt(depth_guess)*star.rs
    max_b = 1+rp_guess/star.rs
    
    print("\nMax. allowed b (1+rp_guess/rs) = {0:0.5f}".format(max_b))
    print("Depth guess = {0:0.5f}. True depth = {1:0.5f}".format(depth_guess, star.depth))
    
    allfixed = np.array([star.eccen, star.smass, star.smass_err, star.rs, star.rs_err, tKep, star.u1, star.u2, max_b])
    
    #initial_simplex = np.tile(first_guess, (len(first_guess)+1,1))
    
    nll = lambda *args: -lnprob(*args)
    
    res0 = op.minimize(nll, [depth_guess/1.1, 0.4, 0.0, star.smass, star.rs, 1.0, period_guess, 0.0], 
                       args=(t, f, 0.0, allfixed), 
                       options={'maxiter':1e5, 'disp': True},
                       method='Nelder-Mead') #options={'maxiter': 1e3, 'disp': True}
    
    res1 = op.minimize(nll, res0["x"], 
                       args=(t, f, ferr, allfixed), 
                       options={'maxiter':1e5, 'disp': True},
                       method='Nelder-Mead')
    
    depth_ml, b_ml, sigma_ml, mass_ml, radius_ml, f0_ml, period_ml, tc_ml = res1["x"]
        
    # Compute the theoretical light curve integrated over a Kepler short-cadence exposure time.
    s = transit.System(transit.Central(mu1=star.u1, mu2=star.u2, mass=mass_ml, radius=radius_ml))
    body = transit.Body(radius=np.sqrt(depth_ml)*radius_ml,period=period_ml, t0=tc_ml,b=np.abs(b_ml),e=star.eccen)
    s.add_body(body)
    t_fit = np.arange(-1, 1, tKep*0.01)
    f_fit = s.light_curve(t_fit, texp=tKep, tol=1e-08, maxdepth=4)
    f_fit = f_fit - 1.0 + f0_ml
    
    if showLC == True:
        fig = plt.figure(figsize=(16,5))
        plt.title("KID"+str(star.kepid)+". Preliminary Fit for Transit "+str(transit_num+1), fontsize = 15)
        plt.plot(t, f, '.', label = 'Kepler data')
        plt.plot(t_fit, f_fit, color='r', lw = 3, label = 'Fit')
        plt.xlabel('Time from midtransit [days]')
        plt.ylabel('Normalized flux')
        plt.xlim([min(t),max(t)]) 
        plt.show(block=False)    
        time.sleep(0.5)
        plt.close()

    #star.duration = body.duration #from eq'n 14 in Winn (2010)
    print("------ Pre-MCMC Results ------")
    print("\tTransit duration [d]:\t Fit={0:0.3f}\tTrue={1:0.3f}".format(body.duration, star.duration))
    print("\tPeriod [days]:\t Fit={0:0.8f}\tTrue={1:0.8f}".format(period_ml,star.P))
    print("\tImpact Parameter:\t Fit={0:0.6f}\tTrue={1:0.6f}".format(b_ml,star.b))
    print("\tDepth:\t Fit={0:0.6f}\tTrue={1:0.6f}".format(depth_ml,star.depth))
    print("\tStellar mass:\t Fit={0:0.3f}\tTrue={1:0.3f}".format(mass_ml,star.smass))
    print("\tStellar radius:\t Fit={0:0.3f}\tTrue={1:0.3f}".format(radius_ml,star.rs))
    print("\tSigma (white noise):\t Fit={0:0.3f}".format(sigma_ml))
    print("\tOut-of-transit flux:\t Fit={0:0.3f}".format(f0_ml))
    
    return(res1["x"])

## MCMC 

In [ ]:
def run_mcmc(star, t, f, ferr, num_trans, fit_results, show_mcmc_res):
    
    depth_ml, b_ml, sigma_ml, mass_ml, radius_ml, f0_ml, period_ml, tc_ml = fit_results
    
    
    #adjust depth a little
    fit_results[0] = fit_results[0]/1.2
    
    rp_guess = np.sqrt(depth_ml)*radius_ml
    max_depth = 1+rp_guess/radius_ml
        
    print("\n------------------ Prepare MCMC ------------------")
    print("Guess impact parameter: {0:0.5f}".format(fit_results[1]))
    print("Guess of Rp [in Rsun]: {0:0.5f}".format(rp_guess))
    print("Max. allowed b (1+rp_guess/rs): {0:0.5f}".format(max_depth))

    allfixed_MCMC = [star.eccen, star.smass, star.smass_err, star.rs, 
                     star.rs_err, tKep, star.u1, star.u2, max_depth]
    
    ndim = len(fit_results)
    
    p0 = [fit_results*(1+1e-8*np.random.randn(ndim)) for i in range(nwalkers)]
    
    sampler = emcee.EnsembleSampler(nwalkers, ndim, lnprob_MCMC, args=(t, f, ferr, allfixed_MCMC))
    
    print("\nRunning burn-in")
    pos, prob, state = sampler.run_mcmc(p0, 200, progress=True);
    sampler.reset() # Reset the chain to remove the burn-in samples.
    print("\n------ Run MCMC -------")
    sampler.run_mcmc(p0, steps, rstate0 = state, progress=True);
    samples = sampler.chain[:, 300:, :].reshape((-1, ndim)) 
    
    """
    ************ BE CAREFUL WITH THE INDEXES OF THE FOLLOWING FOUR LINES!!!!! ************
    This is bc the indexes depend on the list of parameters being fitted.
    """
    period_distribution = samples[:,6]
    samples[:, 2] = np.exp(samples[:, 2])
    planetradsamp = 109.045*np.sqrt(samples[:,0])*samples[:, 4] #in EARTH RAD rad( rp = sqrt(depth)*rs)
    P1 = period_ml*24.0*3600.0 #in sec
    r_asamp = ((3.0*np.pi/(G*P1**2))*((samples[:,4]**3/samples[:, 3])/1408.0))**0.3333

    #Get the 50% percentile and the +- 1 sigma error of the parameters and 2 derived quantities 
    depth_mcmc, b_mcmc, sigma_mcmc, smass_mcmc, rs_mcmc, \
    f0_mcmc, period_mcmc, tc_mcmc = map(lambda v: (v[1], v[2]-v[1], v[1]-v[0]), 
                               zip(*np.percentile(samples, [15.86, 50, 84.14], axis=0)))
    
    v = np.percentile(planetradsamp, [15.86, 50, 84.14], axis=0)
    planetrad_mcmc =(v[1], v[2]-v[1], v[1]-v[0])
    v = np.percentile(r_asamp, [15.86, 50, 84.14], axis=0)
    r_a_mcmc =(v[1], v[2]-v[1], v[1]-v[0])
    
    results_mcmc = [depth_mcmc, b_mcmc, sigma_mcmc, smass_mcmc, rs_mcmc, f0_mcmc, 
                    period_mcmc, tc_mcmc, planetrad_mcmc, r_a_mcmc]
    
    if show_mcmc_res == True:
        print("\n------------------ Post-MCMC Results ------------------")
        print("\nDepth:\n\tTrue = {:0.3f}".format(star.depth))
        print("\tMCMC Fit = {0:0.3f} (+{1:0.3f},-{2:0.3f})".format(depth_mcmc[0],depth_mcmc[1],depth_mcmc[2]))
        print("\nImpact parameter:\n\tTrue = {:0.3f}".format(star.b))
        print("\tMCMC Fit = {0:0.3f} (+{1:0.3f},-{2:0.3f})".format(b_mcmc[0],b_mcmc[1],b_mcmc[2]))
        print("\nPlanet radius [REarth]:\n\tTrue = {:0.3f}".format(star.rp)) #In earth radii due to the conversion done above 
        print("\tMCMC Fit = {0:0.3f} (+{1:0.3f},-{2:0.3f})".format(planetrad_mcmc[0],planetrad_mcmc[1],planetrad_mcmc[2]))
        print("\nPeriod [d]:\n\tTrue = {:0.3f}".format(star.P))
        print("\tMCMC Fit = {0:0.3f} (+{1:0.3f},-{2:0.3f})".format(period_mcmc[0],period_mcmc[1],period_mcmc[2]))
        print("\nInverse of scaled semi-major axis:\n\tTrue = {:0.4f}".format(1/star.dor))
        print("\tMCMC Fit = {0:0.4f} (+{1:0.4f},-{2:0.4f})".format(r_a_mcmc[0],r_a_mcmc[1],r_a_mcmc[2]))
        print("\nStellar radius [Rsun]:\n\tTrue = {:0.3f}".format(star.rs))
        print("\tMCMC Fit = {0:0.3f} (+{1:0.3f},-{2:0.3f})".format(rs_mcmc[0],rs_mcmc[1],rs_mcmc[2]))
        print("\nStellar Mass [Msun]:\n\tTrue = {:0.3f}".format(star.smass))
        print("\tMCMC Fit = {0:0.3f} (+{1:0.3f},-{2:0.3f})".format(smass_mcmc[0],smass_mcmc[1],smass_mcmc[2]))

    #---------- Plot the LC with the MCMC fit ----------
    s = transit.System(transit.Central(mu1=star.u1, mu2=star.u2, mass=smass_mcmc[0], radius=rs_mcmc[0]))
    body = transit.Body(radius=np.sqrt(depth_mcmc[0])*rs_mcmc[0], period=period_mcmc[0],
                        t0=tc_mcmc[0], b=np.abs(b_mcmc[0]), e=star.eccen)
    s.add_body(body)
    t_fit = np.arange(-1.5, 1.5, tKep*0.01)
    f_fit = s.light_curve(t_fit, texp=tKep, tol=1e-08, maxdepth=4)
    f_fit = f_fit - 1.0 + f0_mcmc[0]
    fig_lc = plt.figure(figsize=(15,4))
    plt.title('MCMC Fit for KID'+str(star.kepid))
    plt.plot(t, f, '.', label = 'kepler')
    plt.plot(t_fit, f_fit, 'r', lw = 3, label = 'fit')
    plt.xlabel('Time [d]'); plt.ylabel('Normalized Flux')
    plt.xlim([min(t),max(t)]); 
    
    file_path_fit = '/Users/mbadenas/Documents/Master UAB/Tesis UAB/TFM2018/results/MCMC_fits/'
    store_mcmc_fit = "fit"+str(star.kepid)+"_st"+str(steps)+"_trans"+str(num_trans+1)+"_"+str(period_guess)+"d.png"
    fig_lc.savefig(file_path_fit+store_mcmc_fit)
    plt.close(fig_lc)
        
    #Create a corner plot
    true_params = [star.depth, star.b, None, star.smass, star.rs, 1.0, star.P, 0.0]
    res_labels = ['Depth', 'b', r'$\sigma$', r'$M_{*} [M_{\odot}]$', r'$R_{*} [R_{\odot}]$', 
              r'$f_{0}$', 'P [d]', r'$t_c$']
    
    fig_corner = corner.corner(samples, weights = None, labels = res_labels, quantiles=[0.16, 0.5, 0.84],
                               truths = true_params, show_titles = True, title_args={"fontsize": 12}, 
                               plot_contours=True, truth_color='r', plot_datapoints=True) 
    
    plt.suptitle('Corner Plot for KID'+str(star.kepid), fontsize = 20)
    file_path = '/Users/mbadenas/Documents/Master UAB/Tesis UAB/TFM2018/results/corner/'
    
    store_corner = "corner"+str(star.kepid)+"_st"+str(steps)+"_trans"+str(num_trans+1)+"_"+str(period_guess)+"d.png"
    fig_corner.savefig(file_path+store_corner)
    plt.close(fig_corner)
   
    #Acceptance fraction: as a rule of thumb, the acceptance fraction (af) should be between 0.2 and 0.5. 
    #If af < 0.2 decrease the a parameter. If af > 0.5 increase the a parameter (see emcee.EnsembleSampler input options) 
    af = sampler.acceptance_fraction  
    print("\nMean acceptance fraction:", np.mean(af))
    print("Acceptance fraction: {0:.2f} %".format(100*np.mean(sampler.acceptance_fraction)))
    
    #Create a steps plot
    fig_b, axes = plt.subplots(8, 1, sharex=True, figsize=(8, 9))
    axes[0].plot(sampler.chain[:, :, 0].T, ls='dotted', color = "xkcd:sky blue") 
    axes[0].set_ylabel(r'Depth'); axes[0].axhline(true_params[0], color='red')
    axes[1].plot(sampler.chain[:, :, 1].T,'.', ls='dotted', color="xkcd:sky blue")
    axes[1].set_ylabel(r'$b$'); axes[1].axhline(true_params[1], color='red')
    axes[2].plot((sampler.chain[:, :, 2]).T, ls='dotted', color="xkcd:sky blue")
    axes[2].set_ylabel(r'$\sigma$');
    axes[3].plot((sampler.chain[:, :, 3]).T, ls='dotted', color="xkcd:sky blue")
    axes[3].set_ylabel(r'$M_{*} [M_{\odot}]$'); axes[3].axhline(true_params[3], color='red')
    axes[4].plot((sampler.chain[:, :, 4]).T, ls='dotted', color="xkcd:sky blue")
    axes[4].set_ylabel(r'$R_{*} [R_{\odot}]$'); axes[4].axhline(true_params[4], color='red')
    axes[5].plot((sampler.chain[:, :, 5]).T, ls='dotted', color="xkcd:sky blue")
    axes[5].set_ylabel('$f_{0}$'); axes[5].axhline(true_params[5], color='red')
    axes[6].plot((sampler.chain[:, :, 6]).T, ls='dotted',color="xkcd:sky blue")
    axes[6].set_ylabel("$P [d]$"); axes[6].axhline(true_params[6], color='red')
    axes[7].plot((sampler.chain[:, :, 7]).T, ls='dotted',color="xkcd:sky blue")
    axes[7].set_ylabel("$t_{c}$");
    axes[7].set_xlabel("Step number")
    fig_b.tight_layout(h_pad=0.0)
    filepath = '/Users/mbadenas/Documents/Master UAB/Tesis UAB/TFM2018/results/burnin/'
    store_steps= "steps"+str(star.kepid)+"_st"+str(steps)+"_trans"+str(num_trans+1)+".png"
    fig_b.savefig(filepath+store_steps)
    plt.close(fig_b)
    
    return (period_distribution, results_mcmc)

## Transit Analysis

In [ ]:
amplify

Fit each transit with the new stellar density

In [ ]:
for target in targets:
    print("\n******************************** KID"+str(target.kepid)+" ********************************")
    key = target.kepid
    missed_trans = set(target.ind_missed_transits)
    
    fit_results = []
    period_distributions = []
    MCMC_transits = []

    for i in range(len(target.transit_times)): 
        if i in missed_trans:
            fit_results.append(0)  
            period_distributions.append(0)
            MCMC_transits.append(0)
        else: 
            print("\n*TRANSIT "+str(i+1))

            dt_i = target.ttrans[target.trans_id==i]-target.transit_times[i]
            f_trans_i = target.ftrans[target.trans_id==i]

            oot = np.abs(dt_i)>=(target.duration/2)
            iit = np.abs(dt_i)<(target.duration/2)
            d = 1-np.mean(f_trans_i[iit])
            error_trans_oot = np.std(f_trans_i[oot])
            snr = d/error_trans_oot

            print("\tSignal = {0:0.5f}. Noise (rms) = {1:0.5f}. SNR = {2:0.5f}".format(d, error_trans_oot, snr))
            print("\tOut-of-transit error of the transit LC = {0:0.5f}".format(error_trans_oot))

            if  snr > snr_limit:  
                lc_fit = run_transit_routine(target, dt_i, f_trans_i, 0.1*error_trans_oot, i, True)
                fit_results.append(lc_fit)
            
                period_res, mcmc_res = run_mcmc(target, dt_i, f_trans_i, 0.1*error_trans_oot, i, lc_fit, True)
                period_distributions.append(period_res)
        
                MCMC_transits.append(mcmc_res)

                locs_P_trans = '/Users/mbadenas/Documents/Master UAB/Tesis UAB/TFM2018/results/period_distributions/'
                locs_MCMC = '/Users/mbadenas/Documents/Master UAB/Tesis UAB/TFM2018/results/MCMC_results/'
                
                fname_P_trans = str(target.kepid)+"_st"+str(steps)+"_Pdist_trans"+str(i+1)+"_"+str(period_guess)+"d.txt"
                fname_MCMC = str(target.kepid)+"_st"+str(steps)+"_MCMCres_trans"+str(i+1)+"_"+str(period_guess)+"d.txt"
                
                np.savetxt(locs_P_trans+fname_P_trans, np.transpose(period_res), fmt = "%0.15f")
                np.savetxt(locs_MCMC+fname_MCMC, np.column_stack((id_mcmc, mcmc_res)), 
                               header = header_mcmc, delimiter=" ", fmt="%s")

    target.res_transfit = fit_results
    target.P_distribution = period_distributions
    target.MCMC_transits = MCMC_transits

# Period Analysis

In [ ]:
#colors: https://matplotlib.org/examples/color/named_colors.html

def plot_periods(star):
    bin_num = 15
    fig = plt.figure(figsize=(16,14))
    ax=fig.add_subplot(211)
    for i in range(len(star.full_P_trans)):
        ax.hist(star.full_P_trans[i]['Period'], density = True, bins=30, histtype='step', label='Transit '+str(i+1));
    ax.set_title('KID'+str(star.kepid)+' - Transit Period distribution', fontsize = 20)
    plt.axvline(x = star.P, color = 'r', linewidth = 4, label = 'True Period' )
    ax.set_xlabel(r'$P$ [d]')
    ax.set_ylabel('Frequency')
    if (len(star.transit_times) < 10): ax.legend(loc='best', fontsize = 12)

    ax2 = fig.add_subplot(212)
    ax2.set_title('KID'+str(star.kepid)+' - Full Period Distribution', fontsize = 20)
    ax2.hist(star.full_P['Period'], bins=bin_num*3, density = True, color='wheat', alpha = 0.5) 
    ax2.set_xlabel(r'$P$ [d]')
    ax2.set_ylabel('Frequency')
    plt.axvline(x = star.P, color = 'r', linewidth = 4, label = 'True Period')
    plt.axvline(x = star.full_stats['mode'][0], color = 'b', ls = 'dashed', lw = 3, label = 'Mode')
    plt.axvline(x = star.full_stats['mean'][0], color = 'k', ls = 'dashed', lw = 3, label = 'Mean')
    plt.axvline(x = star.full_stats['median'][0], color = 'r',  ls = 'dashed', lw = 3, label = 'Median')
    plt.axvline(x = star.full_stats['16%'][0], color = 'k', ls = ':', lw = 3, label = '16%')
    plt.axvline(x = star.full_stats['50%'][0], color = 'm', ls = ':', lw = 3, label = '50%')
    plt.axvline(x = star.full_stats['84%'][0], color = 'g', ls = ':', lw = 3, label = '84%')
    ax2.legend(loc = 'best', fontsize = 12)
    plt.show()
    plt.tight_layout()
    
    path_hist = '/Users/mbadenas/Documents/Master UAB/Tesis UAB/TFM2018/results/period_charts/'
    fig.savefig(path_hist+"hist"+str(star.kepid)+".png")
    plt.close(fig)

In [ ]:
def get_statistics(data):
    full_statistics = pd.DataFrame({'mean': data.mean(), 'median': data.median(),'mode': data.mode(),
                        'std': data.std(), 'min': data.min(), 'max': data.max(), 
                   '16%': data.quantile(0.16), '50%': data.quantile(0.5), '84%': data.quantile(0.84)})
    return full_statistics

def find_nearest(array, value):
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    return array[idx]

def closest_statistic(df, P):
    cols = ['Mean', 'Median', 'Mode']
    stats = [df['mean'], df['median'], df['mode']]
    res = find_nearest(stats, P)
    idx_res = np.where(stats == res)[0][0]
    return cols[idx_res]

def show_best_stats(star):
    df = pd.DataFrame({'Statistics': star.best_stats})
    freq = df['Statistics'].value_counts().to_dict()
    colors = ['lightcoral', 'lightskyblue', 'lightgreen']
    #['lightpink', 'lightgreen','lightskyblue']
    
    fig = plt.figure(figsize=(12,4))
    ax = fig.add_subplot(111)
    #ax.pie(freq.values(), labels = freq.keys(), colors = colors, autopct='%1.2f%%', shadow = False, startangle = 90)
    ax.pie(freq.values(), colors = colors, autopct='%1.2f%%', shadow = False, startangle = 90)
    plt.legend(loc ='best', labels = ['%s: %d%%' % (l, s) for l, s in zip(freq.keys(), freq.values())],shadow=True)
    ax.axis('equal')  
    ax.set_title("KID"+str(star.kepid), fontsize = 15)
    left, bottom, width, height = [0.15, 0.5, 0.2, 0.2] # These are in unitless percentages of the figure size. (0,0 is bottom left)
    axins = fig.add_axes([left, bottom, width, height])
    missed_trans = set(star.ind_missed_transits)
    
    for i in range(len(target.transit_times)): 
        if i not in missed_trans:
            axins.plot(star.ttrans[star.trans_id==i]-star.transit_times[i],star.ftrans[star.trans_id==i],'.', 
                       color = 'royalblue', ms = 0.3)
    axins.set_xlabel('Time [d]',fontsize = 8)
    axins.set_ylabel('Flux', fontsize = 8)
    axins.set_title('LC', fontsize = 10)
    plt.tight_layout()
    plt.show()
    
    path_pie = '/Users/mbadenas/Documents/Master UAB/Tesis UAB/TFM2018/results/period_charts/'
    fig.savefig(path_pie+"pie"+str(star.kepid)+".png")
    plt.close(fig)

In [ ]:
path_newRs = '/Users/mbadenas/Documents/Master UAB/Tesis UAB/TFM2018/results/period_distributions/'
os.chdir(path_newRs)

count = 0
save_best_stats = []

for target in targets:
    period_distribution = []
    P_trans_stats = []
    best_statistics = []
    search_files = glob.glob(path_newRs+str(target.kepid)+"*[!_BAD].txt")
    
    if len(search_files) != 0:
        for i in range(len(search_files)):
            filename = search_files[i]
            Ptrans = pd.read_csv(filename, sep = " ", names=['Period'], index_col = None)
            period_distribution.append(Ptrans)
            trans_stats = get_statistics(Ptrans['Period']).iloc[0]
            P_trans_stats.append(trans_stats)
            best_statistics.append(closest_statistic(trans_stats, target.P))
        target.full_P_trans = period_distribution
        target.full_P = pd.concat(period_distribution)
        target.best_stats = best_statistics
        save_best_stats.append(best_statistics)
        
        #Calculate statistical properties 
        target.full_P_trans_stats = P_trans_stats
        target.full_stats = get_statistics(target.full_P['Period'])
        plot_periods(target)
        show_best_stats(target)
        count += 1

In [ ]:
print(count)
save_best_stats = list(itertools.chain.from_iterable(save_best_stats))
print(save_best_stats)

In [ ]:
import seaborn as sns
path_resMCMC = '/Users/mbadenas/Documents/Master UAB/Tesis UAB/TFM2018/results/MCMC_results/'
path_pie = '/Users/mbadenas/Documents/Master UAB/Tesis UAB/TFM2018/results/period_charts/'

for target in targets:
    getmc = glob.glob(path_resMCMC+str(target.kepid)+"*[!_BAD].txt")
    num_trans = 0
    
    if len(getmc) != 0:
        print("\n--------------------------------KID"+str(target.kepid)+"--------------------------------")
        print("True Period:", target.P)
        
        xtrans = []
        oc = []
        
        for i in range(len(getmc)): 
            filename = getmc[i]
            num_trans += 1
            data = {}
            for line in open(filename):
                if line.startswith("#"):
                    continue
                field_name, values = line.split(":")
                data[field_name] = values.split(" ")
            
            df = pd.DataFrame(data)
            
            est_p = float(df['Orbital Period (P, [days])'][1]) #this is the median result for each transit
            xtrans.append(i)
            oc.append(est_p-target.P)
        
        #print(oc)
        print("Standard deviation OC [d]:", np.std(oc))
        print("Mean OC [d]:", np.mean(oc))
        print("Mean Estimated Period [d]:", np.mean(est_p))
        
        textstr = 'Mean O-C = {0:.2f} d\nstd O-C = {1:.2f} d\nEstimated Period = {2:.2f} d'.format(np.mean(oc),
                                                                                                np.std(oc), 
                                                                                                np.mean(est_p))

        fig = plt.figure(figsize=(10,5))
        ax = fig.add_subplot(111)
        ax.axhline(y = 0, c = 'r', ls= 'dotted', lw=3, label = 'True Period ({0:.2f} d)'.format(target.P))
        ax.scatter(xtrans, oc, s = 12)
        ax.set_xlabel('Transit number')
        ax.set_ylabel('O-C')
        ax.set_title('Residuals Distribution')
        ax.annotate(textstr, xy=(0.25, 0.8), xycoords='axes fraction',
                    horizontalalignment='right', verticalalignment='bottom', backgroundcolor='wheat',alpha=.85)
        plt.legend()
        plt.savefig(path_pie+'OC'+str(target.kepid)+".png")
        plt.show()
 

PLanet radius vs. orbital period (fig 7 scatter plot http://iopscience.iop.org/article/10.3847/0004-637X/822/2/86/pdf)

keylist = set(list(ugly_stars.keys()))
bad_transits = list(ugly_stars.values())
print(keylist)
#print(bad_transits)
print(ugly_stars)